# Build an Agent

LangChain supports the creation of [agents](/docs/concepts/agents), or systems that use [LLMs](/docs/concepts/chat_models) as reasoning engines to determine which actions to take and the inputs necessary to perform the action.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via [tool-calling](/docs/concepts/tool_calling).

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

### Load environment variables

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [5]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("openai:gpt-5-nano", temperature=1)
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [6]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I live in SF.
================================== Ai Message ==================================

Hi Bob! Nice to meet you. How can I help today? Since you’re in San Francisco, I can tailor things to the Bay Area. A few ideas:
- Local weather, traffic, and transit updates
- Restaurant or cafe recommendations near your neighborhood
- Events, things to do, or weekend ideas in SF
- Real estate or rental tips for the city
- SF news or safety alerts

Want to tell me your neighborhood (e.g., Mission, SoMa) or a topic you’re interested in?


In [7]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_gbm1zxsj9oSF7NhfF6Gc8DFi)
 Call ID: call_gbm1zxsj9oSF7NhfF6Gc8DFi
  Args:
    query: San Francisco current weather
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "San Francisco current weather", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=August", "title": "San Francisco weather in August 2025 | Weather25.com", "content": "weather25.com\nSearch\nweather in United States\nRemove from your favorite locations\nAdd to my locations\nShare\nweather in United States\n\n# San Francisco weather in August 2025\n\nCloudy\nOvercast\nMist\nClear\nClear\nSunny\nPa

## Setup

### Jupyter Notebook

This guide (and most of the other guides in the documentation) uses [Jupyter notebooks](https://jupyter.org/) and assumes the reader is as well. Jupyter notebooks are perfect interactive environments for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc), and observing these cases is a great way to better understand building with LLMs.

This and other tutorials are perhaps most conveniently run in a Jupyter notebook. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

To install LangChain run:

In [9]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite --break-system-packages

  Attempting uninstall: langgraph-sdk
    Found existing installation: langgraph-sdk 0.2.0
    Uninstalling langgraph-sdk-0.2.0:
      Successfully uninstalled langgraph-sdk-0.2.0
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.6.4
    Uninstalling langgraph-0.6.4:
      Successfully uninstalled langgraph-0.6.4
Note: you may need to restart the kernel to use updated packages.


For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

### Tavily

We will be using [Tavily](/docs/integrations/tools/tavily_search) (a search engine) as a tool.
In order to use it, you will need to get and set an API key:

```bash
export TAVILY_API_KEY="..."
```

Or, if in a notebook, you can set it with:

```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

## Define tools

We first need to create the tools we want to use. Our main tool of choice will be [Tavily](/docs/integrations/tools/tavily_search) - a search engine. We can use the dedicated [langchain-tavily](https://pypi.org/project/langchain-tavily/) [integration package](/docs/concepts/architecture/#integration-packages) to easily use Tavily search engine as tool with LangChain.


In [10]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756313356, 'localtime': '2025-08-27 09:49'}, 'current': {'last_updated_epoch': 1756313100, 'last_updated': '2025-08-27 09:45', 'temp_c': 18.9, 'temp_f': 66.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 230, 'wind_dir': 'SW', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 73, 'cloud': 50, 'feelslike_c': 18.9, 'feelslike_f': 66.0, 'windchill_c': 14.0, 'windchill_f': 57.2, 'heatindex_c': 14.3, 'heatindex_f': 57.7, 'dewpoint_c': 13.4, 'de

:::tip

In many applications, you may want to define custom tools. LangChain supports custom
tool creation via Python functions and other means. Refer to the
[How to create tools](/docs/how_to/custom_tools/) guide for details.

:::

## Using Language Models

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably.

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-5-nano", temperature=1)

You can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [12]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help today? I can explain concepts, answer questions, help with writing or coding, brainstorm ideas, plan something, or just chat. Tell me what you’d like to do or share a topic, and I’ll dive in.'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [13]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [14]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hi there! How can I help today?

I can:
- Explain concepts or answer questions
- Draft or edit emails, essays, or code
- Brainstorm ideas or plans (travel, projects, etc.)
- Summarize articles or documents
- Help with debugging or learning something new

Tell me what you’re aiming to do, or share a topic you’re curious about. If you’re unsure, here are quick prompts you can try:
- Explain [concept] in simple terms
- Draft a polite email about [topic]
- Plan a [duration]-day itinerary for [place]
- Help me debug this [language] code snippet

What would you like to start with?

Tool calls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [15]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current weather San Francisco', 'include_domains': ['weather.gov', 'noaa.gov', 'weather.com', 'accuweather.com', 'forecast.weather.gov'], 'search_depth': 'advanced', 'include_favicon': False, 'time_range': 'day'}, 'id': 'call_ne6JzkdswrSrBm9DSxx8DMaa', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create the agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using [LangGraph](/docs/concepts/architecture/#langgraph) to construct the agent.
Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.


Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

We can now run the agent with a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions). Note that the agent will return the **final** state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [17]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hi there! How can I help today? I can answer questions, brainstorm ideas, draft emails or documents, explain concepts, help with coding, plan trips, summarize articles, and more. What would you like to do?


In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Let's now try it out on an example where it should be invoking the tool

In [18]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_XFtj6Hf8scJvwxWqSbnWIgMT)
 Call ID: call_XFtj6Hf8scJvwxWqSbnWIgMT
  Args:
    query: San Francisco weather current
    include_domains: ['weather.gov', 'noaa.gov', 'weather.com', 'accuweather.com', 'timeanddate.com', 'bbc.com']
================================= Tool Message =================================
Name: tavily_search

{"query": "San Francisco weather current", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, California", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756313790, 'localtime': '2025-08-27 09:56'}

We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

## Streaming Messages

We've seen how the agent can be called with `.invoke` to get  a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [19]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_tRLqro4sx4upz6UtWfBqqz3n)
 Call ID: call_tRLqro4sx4upz6UtWfBqqz3n
  Args:
    query: San Francisco weather today
    include_domains: ['weather.gov', 'weather.com', 'accuweather.com', 'noaa.gov']
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "San Francisco weather today", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://forecast.weather.gov/MapClick.php?lat=37.9036&lon=-122.5436&unit=0&lg=english&FcstType=graphical", "title": "Hourly Weather Forecast - National Weather Service", "content": "| 48-Hour Period Starting: 9am Wed, Aug 27 202510am Wed, Aug 27 202511am Wed, Aug 27 202512pm Wed, Aug 27 20251pm Wed, Aug 27 20252pm Wed, Aug 27 

## Streaming tokens

In addition to streaming back messages, it is also useful to stream back tokens.
We can do this by specifying `stream_mode="messages"`.


::: note

Below we use `message.text()`, which requires `langchain-core>=0.3.37`.

:::

In [20]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

I| found| a| few| weather| sources| from| the| recent| search|.| Do| you| want| current| conditions|,| a| forecast|,| or| both| for| a| specific| SF| location|?| Here| are| the| results| I| found|:

|-| National| Weather| Service| Marine| Forecast| for| Central| California| Coast| (|F|Z|US|56| KM|TR|)
| | -| Covers| SF| Bay| area| coastal| waters| and| nearby| bays|
| | -| Includes| wind|,| fog|,| and| marine| conditions| for| today| and| tonight|
| | -| Useful| if| you|’re| boating| or| need| offshore|/|shore|line| conditions|

|-| National| Weather| Service|/N|DBC| Coastal| Waters| Forecast|s| for| SF| Bay|
| | -| SF| Bay| South| of| the| Bay| Bridge| and| SF| Bay| North| of| the| Bay| Bridge|
| | -| Wind| forecasts| and| marine| conditions|

|-| Acc|u|Weather| page| for| South| San| Francisco| (|ZIP| |940|80|)
| | -| Monthly| forecast| page| (|not| a| current| conditions| page|)

|If| you| want|,| I| can| pull| live| current| conditions| and| a| |7|-day| forecast| for|:
|-| San| Fra

## Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [21]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [22]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [23]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hi Bob! Nice to meet you. How can I help today? If you’re not sure, here are a few ideas:
- Answer a question or explain something you’re curious about
- Help plan a trip, project, or schedule
- Debug code or learn a programming concept
- Write or summarize documents, emails, or reports
- Get quick facts or a deeper dive into a topic

Tell me what you’re working on or just say “surprise me,” and we’ll take it from there.


In [24]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob. Nice to meet you, Bob! Would you like me to call you Bob or a nickname instead, and what would you like help with today?


Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If you want to start a new conversation, all you have to do is change the `thread_id` used

In [ ]:
# highlight-next-line
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent.
We've then shown how to stream back a response - not only with the intermediate steps, but also tokens!
We've also added in memory so you can have a conversation with them.
Agents are a complex topic with lots to learn!

For more information on Agents, please check out the [LangGraph](/docs/concepts/architecture/#langgraph) documentation. This has it's own set of concepts, tutorials, and how-to guides.